In [1]:
import pandas as pd
import numpy as np
from functions import *

%load_ext autoreload
%autoreload 2

### STILL MISSING TO ADD BETA

In [2]:
trade = pd.read_csv('data/trade.csv', header=[0,1], index_col=0)
stocks = pd.read_csv('data/stocks.csv')
dk_industry = pd.read_csv('data/dk_industry.csv')
financials = pd.read_csv('data/financials.csv')
beta = pd.read_csv('data/beta.csv')

# display(trade.head())
# display(stocks.head())
# display(dk_industry.head())

In [3]:
# offsets
quarterly_offset = 1 # quarterly data is 2 months behind end of quarter
annual_offset = 6 # annual data is 6 months behind publication

In [4]:
# convert the index to datetime (the index holds the dates)
trade.index = pd.to_datetime(trade.index)
trade = trade.stack(level=0).reset_index()


# rename columns to have a proper
trade.rename(columns={'level_1': 'ticker'}, inplace=True)

# Data Cleaning and Sorting
trade.drop_duplicates(inplace=True)
trade.dropna(inplace=True) # happens if there was one data point the first day of a given ticker but not the rest of the values (e.g. trade values but no ask or bid)
trade.sort_values(['ticker', 'timestamp'], inplace=True)
trade.reset_index(drop=True, inplace=True)

# display(trade)


/var/folders/yw/jsw5n53s1cb1s2q6tt0msrm00000gn/T/ipykernel_36064/2362051604.py:3: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  trade = trade.stack(level=0).reset_index()


In [5]:
df = pd.merge(trade, stocks[['ticker','shares','NACE']], how='left', on=['ticker'])
# display(df)

### Add industry data for DK

In [6]:
dk_industry['timestamp'] = pd.to_datetime(dk_industry['timestamp'])
# join stocks and dk_industry on 'NACE industry' 
industry = stocks[['ticker','NACE']].merge(dk_industry, how='left', on='NACE')

# adjust the timestamp to be 2 months behind
industry['timestamp'] = industry['timestamp'] + pd.DateOffset(months=2+quarterly_offset)
industry['timestamp'] = (
    industry['timestamp']
      .dt.to_period('M')
      .dt.to_timestamp('M')
)

industry = industry.drop(columns=['NACE'])

# display(industry)

In [7]:
df = pd.merge(df, industry, how='left', on=['timestamp', 'ticker'])

#ffil the industry values
for col in industry.columns[2:]:
    df[col] = df[col].groupby(df['ticker']).ffill()

# display(df)


### Add financial data

In [8]:
financials['timestamp'] = pd.to_datetime(financials['timestamp'])

financials['timestamp1'] = financials['timestamp']

financials['timestamp'] = financials['timestamp'] + pd.DateOffset(months=annual_offset)
financials['timestamp'] = (
    financials['timestamp']
      .dt.to_period('M')
      .dt.to_timestamp('M')
)

# expand the dataset
financials = (
    financials
        # sort data and find the next timestamp
      .sort_values(['ticker','timestamp'])
      .assign(
        next_fye   = lambda df: df.groupby('ticker')['timestamp'].shift(-1),
        plus_12m   = lambda df: df['timestamp'] + pd.DateOffset(months=12),
        period_end = lambda df: pd.to_datetime(np.where(
                          (df.next_fye - df.timestamp).abs()
                            < 
                          (df.plus_12m   - df.timestamp).abs(),
                          df.next_fye,
                          df.plus_12m
                        ))
      )
      # expand the data
      .assign(timestamp = lambda df: df.apply(expand_monthly, axis=1))
      .explode('timestamp')
      .drop(columns=['next_fye','plus_12m','period_end'])
      .reset_index(drop=True)
)

# display(financials)

In [9]:
# join df and financials
df = pd.merge(df, financials, how='left', on=['timestamp', 'ticker'])

# display(df)

### Add beta

In [10]:
beta['timestamp'] = pd.to_datetime(beta['timestamp'])

# adjust to be last trading day of the month only and set to end of month
monthly_beta = (
    beta
    .groupby(
        ['ticker', pd.Grouper(key='timestamp', freq='ME')],
         as_index=False
    )['beta']
    .last()
)

# display(monthly_beta)

In [ ]:
# join df and beta
df = pd.merge(df, monthly_beta, how='left', on=['timestamp', 'ticker'])

# display(df)

,timestamp,ticker,ask,bid,adjclose,high,low,open,volume,turnover,...,revenue,rnd,cash,grossprofit,opinc,cashflow,debt,currentassets,timestamp1,beta
0,2001-01-31,AAB.CO,4183.493029,4026.612041,4026.612041,4183.493029,2614.683143,2771.564132,6.915369e+02,2.255122e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,-0.704363
1,2001-02-28,AAB.CO,3765.143726,3660.556401,3765.143726,4235.786692,3399.088086,4131.199367,5.061034e+02,2.047656e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.445778
2,2001-03-31,AAB.CO,3660.556401,3529.822244,3503.675412,3765.143726,3451.381749,3765.143726,1.172800e+02,4.385660e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.602796
3,2001-04-30,AAB.CO,3346.794423,3189.913435,3189.913435,4183.493029,3189.913435,4183.493029,1.263059e+02,4.467180e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.378161
4,2001-05-31,AAB.CO,3137.619772,3006.885615,2876.151458,3399.088086,2876.151458,3137.619772,1.653355e+02,5.048560e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.260398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49878,2024-12-31,ZELA.CO,716.500000,715.500000,715.500000,819.000000,587.000000,728.000000,4.138727e+06,2.995576e+09,...,342788000.0,666333000.0,1.633057e+09,323614000.0,-707826000.0,-678878000.0,119230000.0,1.779788e+09,2023-12-31,0.923565
49879,2025-01-31,ZELA.CO,738.000000,737.000000,735.500000,808.000000,682.500000,718.500000,3.538390e+06,2.572094e+09,...,342788000.0,666333000.0,1.633057e+09,323614000.0,-707826000.0,-678878000.0,119230000.0,1.779788e+09,2023-12-31,0.966188
49880,2025-02-28,ZELA.CO,665.000000,664.000000,663.000000,788.000000,657.000000,714.500000,3.876841e+06,2.801496e+09,...,342788000.0,666333000.0,1.633057e+09,323614000.0,-707826000.0,-678878000.0,119230000.0,1.779788e+09,2023-12-31,0.929932
49881,2025-03-31,ZELA.CO,520.500000,519.500000,517.000000,725.000000,486.400000,664.500000,1.085072e+07,6.399725e+09,...,342788000.0,666333000.0,1.633057e+09,323614000.0,-707826000.0,-678878000.0,119230000.0,1.779788e+09,2023-12-31,0.813600


### Create variables

In [12]:
df['target'] = df.groupby('ticker')['adjclose'].transform(lambda x: x.pct_change(periods=1))
df['target'] = df['target'].shift(-1) # shift the target by 1 month

# momentum
momentum_periods = {'mom1m': 1, 'mom3m': 3, 'mom6m': 6, 'mom12m': 12}

# compute percentage change over each period for each ticker separately.
for feature_name, period in momentum_periods.items():
    df[feature_name] = df.groupby('ticker')['adjclose'].transform(lambda x: x.pct_change(periods=period))

# bid-ask spread in percentage
df['baspread'] = (df['ask']-df['bid'])/df['ask']

# display(df)

In [ ]:
# drop if mom12m or target is nan (first 12 months and last month for each ticker)
df.dropna(subset=['mom12m','target'], inplace=True)

# drop if any nan
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# display(df)

,timestamp,ticker,ask,bid,adjclose,high,low,open,volume,turnover,...,debt,currentassets,timestamp1,beta,target,mom1m,mom3m,mom6m,mom12m,baspread
0,2002-01-31,AAB.CO,2719.270469,2614.683143,2614.683143,2771.564132,2562.389480,2614.683143,1.063991e+02,2.950000e+05,...,2855000.0,5.657500e+07,2000-12-31,0.077048,0.120000,0.000000,-0.074074,-0.166667,-0.350649,0.038462
1,2002-02-28,AAB.CO,2928.445121,2876.151458,2928.445121,2928.445121,2562.389480,2719.270469,1.944595e+02,5.280000e+05,...,2855000.0,5.657500e+07,2000-12-31,0.117701,0.125000,0.120000,0.076923,-0.081967,-0.222222,0.017857
2,2002-03-31,AAB.CO,3294.500761,3189.913435,3294.500761,3399.088086,2928.445121,2928.445121,1.694852e+02,5.560000e+05,...,2855000.0,5.657500e+07,2000-12-31,0.093670,-0.063492,0.125000,0.260000,0.125000,-0.059701,0.031746
3,2002-04-30,AAB.CO,3085.326109,2928.445121,3085.326109,3189.913435,2928.445121,3189.913435,1.992593e+02,6.280000e+05,...,2855000.0,5.657500e+07,2000-12-31,0.111161,-0.169492,-0.063492,0.180000,0.092593,-0.032787,0.050847
4,2002-05-31,AAB.CO,2614.683143,2457.802155,2562.389480,2928.445121,2510.095818,2928.445121,6.842129e+01,1.960000e+05,...,2855000.0,5.657500e+07,2000-12-31,0.101388,-0.040816,-0.169492,-0.125000,-0.057692,-0.109091,0.060000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45619,2024-11-30,ZELA.CO,730.000000,729.000000,730.000000,893.000000,672.500000,789.500000,4.650485e+06,3.587543e+09,...,119230000.0,1.779788e+09,2023-12-31,1.155276,-0.019863,-0.076534,-0.176537,0.141517,1.172619,0.001370
45620,2024-12-31,ZELA.CO,716.500000,715.500000,715.500000,819.000000,587.000000,728.000000,4.138727e+06,2.995576e+09,...,119230000.0,1.779788e+09,2023-12-31,0.923565,0.027952,-0.019863,-0.119926,-0.198319,0.917203,0.001396
45621,2025-01-31,ZELA.CO,738.000000,737.000000,735.500000,808.000000,682.500000,718.500000,3.538390e+06,2.572094e+09,...,119230000.0,1.779788e+09,2023-12-31,0.966188,-0.098572,0.027952,-0.069576,-0.209140,0.557603,0.001355
45622,2025-02-28,ZELA.CO,665.000000,664.000000,663.000000,788.000000,657.000000,714.500000,3.876841e+06,2.801496e+09,...,119230000.0,1.779788e+09,2023-12-31,0.929932,-0.220211,-0.098572,-0.091781,-0.252115,-0.001506,0.001504


In [14]:
# save the data
df.to_csv('data/data.csv', index=False)

### Troubleshooting missing variable (only  ~200 rows out of 45k)
NB to see anything comment out the dropna ealier

In [15]:
# # drop if mom12m is nan
# df.dropna(subset=['mom12m','target'], inplace=True)

# nan_tickers = df[df.isna().any(axis=1)][['ticker','timestamp1','netinc', 'assets', 'revenue', 'rnd', 'cash',
#        'grossprofit', 'opinc', 'cashflow', 'debt', 'currentassets']].drop_duplicates()
# nan_tickers = nan_tickers.sort_values(['ticker','timestamp1'])
# # nan_tickers['year'] = nan_tickers['timestamp1'].dt.year
# nan_tickers = nan_tickers[['ticker','timestamp1','netinc', 'assets', 'revenue', 'rnd', 'cash',
#        'grossprofit', 'opinc', 'cashflow', 'debt', 'currentassets']].drop_duplicates().reset_index(drop=True)
# display(nan_tickers)

In [16]:
# display(financials[(financials['ticker'] == 'INVb.CO^F05') & (financials['timestamp1'].dt.year == 2002)])
# display(financials[(financials['ticker'] == 'INVb.CO^F05') & (financials['timestamp1'].dt.year == 2017)])
# display(financials[(financials['ticker'] == 'INVb.CO^F05')].head(60))